### Настройка окружения

In [ ]:
! pip install torchkge -U
!pip install dill

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 1.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.1 MB/s eta 0:00:00


In [ ]:
# переходим в нашу директорию
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)
%cd '/content/drive/MyDrive/joie'

Mounted at /content/drive/
/content/drive/MyDrive/joie


In [ ]:
# для импорта модулей связанных с joie
import sys
sys.path.insert(0,'joie')

In [ ]:
# import torchkge
import torch
from torchkge.data_structures import KnowledgeGraph,SmallKG
import pandas as pd
import numpy as np
from torch import cuda

from joie.models import *
from joie.evaluation import *
from joie.trainer import *
from joie.utils import *

from torchkge.utils import MarginLoss
from torchkge.models.bilinear import DistMultModel, HolEModel
from torchkge.models.translation import TransEModel
from torchkge.evaluation import LinkPredictionEvaluator

/usr/local/lib/python3.10/dist-packages/torchkge/utils/data_redundancy.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Загрузка данных

In [ ]:
datasets = {data: 
            {
              type_kg : 
                to_structure_kg(f'data/{data}/{data}_{type_kg}.txt') for type_kg in ['insnet','ontonet','instype']
             } 
            for data in ['yago']
          }

In [ ]:
# получение test
kg_insnet_test =  to_structure_kg(f'data/yago/yago_insnet_small_test.txt')
kg_instype_test = to_structure_kg(f'data/yago/yago_InsType_small_test.txt')
kg_ontonet_test = to_structure_kg(f'data/yago/yago_ontonet_test.txt')

### JOIE TransE Cross Groupping

In [ ]:
joie_transe_cg = JOIE(kg_instance = datasets['db']['insnet'], 
                      kg_ontology = datasets['db']['ontonet'], 
                      kg_ins_type = datasets['db']['instype'],
    				entity_dim = 200, concept_dim = 200,
    				intra_view_model = 'transe', cross_view_model = 'cg',
    				margin_intra = 0.5, margin_cross = 0.5,**{'lr_instance':0.0005,'lr_ontology':0.0005,'lr_cross':0.0005})

trainer = TrainerJoie(joie = joie_transe_cg,
                       kg_instance = datasets['db']['insnet'],
                       kg_ontology = datasets['db']['ontonet'], 
                       kg_instype = datasets['yago']['instype'],
        n_epochs = 100, n_batches = 300,alpha = 1, omega = 1,sampling_type='bern', use_cuda='all')


In [ ]:
trainer.run(300)

  0%|          | 0/200 [00:00<?, ?epoch/s]

In [ ]:
trainer.plot_train_process()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
results_transe = joie_transe_cg.joie_performance(kg_insnet_test,
                                             kg_ontonet_test,
                                             kg_instype_test,64,100)

In [ ]:
results_transe

### JOIE DistMult Cross Groupping

In [ ]:
joie_distmult_cg = JOIE(kg_instance = datasets['yago']['insnet'], 
                        kg_ontology = datasets['yago']['ontonet'], 
                        kg_ins_type = datasets['yago']['instype'],
    				entity_dim = 200, concept_dim = 200,
    				intra_view_model = 'distmult', cross_view_model = 'cg',
    				margin_intra = 0.5, margin_cross = 0.5,**{'lr_instance':0.0005,'lr_ontology':0.0005,'lr_cross':0.0005})

trainer = TrainerJoie(joie = joie_distmult_cg,
                       kg_instance = datasets['db']['insnet'],
                       kg_ontology = datasets['db']['ontonet'], 
                       kg_instype = datasets['yago']['instype'],
        n_epochs = 100, n_batches = 300,alpha = 1, omega = 1,sampling_type='bern', use_cuda='all')


In [ ]:
trainer.run(120)
trainer.plot_train_process()
torch.cuda.empty_cache()

In [ ]:
results_distmult = joie_distmult_cg.joie_performance(kg_insnet_test,
                                             kg_ontonet_test,
                                             kg_instype_test,64,100)
results_distmult

### JOIE Hole Cross Groupping

In [ ]:
joie_hole_cg = JOIE(kg_instance = datasets['yago']['insnet'], 
                    kg_ontology = datasets['yago']['ontonet'], 
                    kg_ins_type = datasets['yago']['instype'],
    				entity_dim = 200, concept_dim = 200,
    				intra_view_model = 'hole', cross_view_model = 'cg',
    				margin_intra = 0.5, margin_cross = 0.5,**{'lr_instance':0.0005,'lr_ontology':0.0005,'lr_cross':0.0005})

trainer = TrainerJoie(joie = joie_hole_cg,
                       kg_instance = datasets['db']['insnet'],
                       kg_ontology = datasets['db']['ontonet'], 
                       kg_instype = datasets['yago']['instype'],
        n_epochs = 100, n_batches = 300,alpha = 1, omega = 1,sampling_type='bern', use_cuda='all')


In [ ]:
trainer.run(120)
trainer.plot_train_process()
torch.cuda.empty_cache()

In [ ]:
results_hole = joie_hole_cg.joie_performance(kg_insnet_test,
                                             kg_ontonet_test,
                                             kg_instype_test,64,100)
results_hole

### JOIE Rescal Cross Groupping

In [ ]:
joie_complex_cg = JOIE(kg_instance = datasets['yago']['insnet'], 
                      kg_ontology = datasets['yago']['ontonet'], 
                      kg_ins_type = datasets['yago']['instype'],
    				entity_dim = 200, concept_dim = 200,
    				intra_view_model = 'hole', cross_view_model = 'cg',
    				margin_intra = 0.5, margin_cross = 0.5,**{'lr_instance':0.0005,'lr_ontology':0.0005,'lr_cross':0.0005})

trainer = TrainerJoie(joie = joie_complex_cg,
                       kg_instance = datasets['db']['insnet'],
                       kg_ontology = datasets['db']['ontonet'], 
                       kg_instype = datasets['yago']['instype'],
        n_epochs = 100, n_batches = 300,alpha = 1, omega = 1,sampling_type='bern', use_cuda='all')


In [ ]:
trainer.run(120)
trainer.plot_train_process()
torch.cuda.empty_cache()

In [ ]:
results_complex = joie_complex_cg.joie_performance(kg_insnet_test,
                                             kg_ontonet_test,
                                             kg_instype_test,64,100)
results_complex